# Tier 2: RILA Buffer vs Floor

**Learning Objectives:**
- Understand the critical difference between buffer and floor
- See the payoff profiles for each
- Learn the option replication

**Key Distinction [T1]:**
- **Buffer**: Insurer absorbs FIRST X% of losses
- **Floor**: Insurer covers losses BEYOND X%

**Duration:** ~20 minutes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. The Critical Difference

| Protection | Index = -25% | 10% Level | Client Loss |
|------------|--------------|-----------|-------------|
| **Buffer** | -25% | 10% buffer | **15%** (25% - 10%) |
| **Floor** | -25% | -10% floor | **10%** (capped at floor) |

In [ ]:
def rila_buffer(index_return, buffer, cap):
    """Buffer: absorbs FIRST X% of losses."""
    if index_return >= 0:
        return min(index_return, cap)
    else:
        loss = abs(index_return)
        if loss <= buffer:
            return 0.0  # Buffer absorbs all
        else:
            return -(loss - buffer)  # Client bears excess

def rila_floor(index_return, floor, cap):
    """Floor: limits maximum loss to X%."""
    if index_return >= 0:
        return min(index_return, cap)
    else:
        return max(index_return, -floor)  # Loss capped at floor

In [ ]:
# Parameters
buffer_level = 0.10  # 10% buffer
floor_level = 0.10   # 10% floor
cap = 0.15           # 15% cap

# Range of returns
returns = np.linspace(-0.40, 0.30, 100)

# Payoffs
buffer_payoffs = [rila_buffer(r, buffer_level, cap) for r in returns]
floor_payoffs = [rila_floor(r, floor_level, cap) for r in returns]

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(returns * 100, np.array(returns) * 100, 'k--', alpha=0.5, label='Index')
plt.plot(returns * 100, np.array(buffer_payoffs) * 100, 'b-', linewidth=2, label='Buffer (10%)')
plt.axhline(y=0, color='gray', linestyle=':')
plt.xlabel('Index Return (%)')
plt.ylabel('Credited Return (%)')
plt.title('BUFFER Protection\n(Insurer absorbs FIRST 10%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(returns * 100, np.array(returns) * 100, 'k--', alpha=0.5, label='Index')
plt.plot(returns * 100, np.array(floor_payoffs) * 100, 'r-', linewidth=2, label='Floor (-10%)')
plt.axhline(y=0, color='gray', linestyle=':')
plt.axhline(y=-10, color='red', linestyle=':', alpha=0.5)
plt.xlabel('Index Return (%)')
plt.ylabel('Credited Return (%)')
plt.title('FLOOR Protection\n(Max loss limited to 10%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Comparison Table

In [ ]:
test_returns = [-0.40, -0.25, -0.15, -0.10, -0.05, 0.0, 0.10, 0.20]

print(f"{'Index':>10} {'Buffer':>10} {'Floor':>10} {'Difference':>12}")
print("-" * 45)

for r in test_returns:
    b = rila_buffer(r, buffer_level, cap)
    f = rila_floor(r, floor_level, cap)
    diff = abs(b - f)
    print(f"{r:>10.1%} {b:>10.1%} {f:>10.1%} {diff:>12.1%}")

## 3. Option Replication [T1]

| Protection | Replication | Cost Profile |
|------------|-------------|---------------|
| **Buffer** | Long ATM put - Short OTM put | Put spread (cheaper) |
| **Floor** | Long OTM put | Single put (more expensive) |

## 4. Key Takeaways

1. **Buffer** protects at the BOTTOM (absorbs first losses)
2. **Floor** protects at a LEVEL (caps maximum loss)
3. **They give DIFFERENT payoffs** for the same index return
4. **Hedging is different** — don't confuse the option structures!

## Common Mistake

Using floor mechanics when buffer is specified (or vice versa) leads to:
- Wrong client payoffs
- Wrong hedging costs
- Unhedged exposure